In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import result as resultMd
import sampling as sp
reload(fd)
reload(sd)
reload(di)
reload(resultMd)
reload(sp)

print('done')

In [ ]:
import pandas as pd

dfTrx = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrx.head(5)

# Sampling in Python

In [ ]:
from sklearn.model_selection import train_test_split

predictors = fd.getPredictors(dfTrx)
TEST_SIZE = 0.20 # test size using_train_test_split
RANDOM_STATE = 0
x_train0, x_test, y_train0, y_test = train_test_split(dfTrx[predictors], dfTrx['Class'], test_size = TEST_SIZE, 
                                                        stratify= dfTrx['Class'],
                                                        random_state = RANDOM_STATE)

x_train= x_train0.copy()
x_test = x_test.copy()



# Over sampling
* randomOverSampling
* smoteOverSampling

# Under sampling
* randomUnderSampling
* NearMiss
* EditedNearestNeighbours (very slow)
* NeighbourhoodCleaningRule
* oneSidedSelection
* tomekLinks (very slow 33 minutes)

# Combined approach
* Combined random over and underSampling


# Over sampling

In [ ]:
x_train, y_train = sp.randomOverSampling(x_train0, y_train0,rateOverSampling=5)
sp.draw2DPCAScaterPlot(x_train0,y_train0,x_train,y_train)
#sp.saveTrainData(x_train,y_train,"TempTrainRandomOverSampling.pkl")
#x_train2, y_train2=sp.loadTrainData("TempTrainRandomOverSampling.pkl")
#sp.draw2DPCAScaterPlot(x_train0,y_train0,x_train2,y_train2)

In [ ]:

x_train, y_train = sp.smoteOverSampling(x_train0, y_train0,rateOverSampling=5)

sp.draw2DPCAScaterPlot(x_train0,y_train0,x_train,y_train)


# Under sampling

In [ ]:
x_train, y_train = sp.randomUnderSampling(x_train0, y_train0,rate=1/100)
sp.draw2DPCAScaterPlot(x_train0,y_train0,x_train,y_train)

In [ ]:
x_train, y_train = sp.nearMissV1UnderSampling(x_train0, y_train0,rate=1/100)

sp.draw2DPCAScaterPlot(x_train0,y_train0,x_train,y_train)


In [ ]:
x_train, y_train = sp.nearMissV2UnderSampling(x_train0, y_train0,rate=1/100)

sp.draw2DPCAScaterPlot(x_train0,y_train0,x_train,y_train)

In [ ]:
x_train, y_train = sp.oneSidedSelectionUnderSampling(x_train0, y_train0)

sp.draw2DPCAScaterPlot(x_train0,y_train0,x_train,y_train)

In [ ]:
# 33 minutes 1998s

#After
#Class
#0    846839
#1      1040

#After
#Class
#0    846493
#1      1040

x_train, y_train = sp.tomekLinksUnderSampling(x_train0, y_train0)

sp.draw2DPCAScaterPlot(x_train0,y_train0,x_train,y_train)
sp.saveTrainData(x_train,y_train,"TempTrainTomekLinksUnderSampling.pkl")

In [ ]:
# Duration 2103.2 s 

x_train, y_train = sp.neighbourhoodCleaningRuleUnderSampling(x_train0, y_train0,rate=1/100)

sp.draw2DPCAScaterPlot(x_train0,y_train0,x_train,y_train)

# combined approach

In [ ]:
x_train, y_train = sp.randomOverUnderSampling(x_train0, y_train0)
sp.draw2DPCAScaterPlot(x_train0,y_train0,x_train,y_train)